In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
clean_df = pd.read_csv('clean_df_1Mar2024.csv')

In [3]:
data=clean_df

# Convert 'Date_Rptd' and 'DATE_OCC' to datetime
# data['Date_Rptd'] = pd.to_datetime(data['Date_Rptd'], format='%m/%d/%Y %I:%M:%S %p')
# data['DATE_OCC'] = pd.to_datetime(data['DATE_OCC'], format='%Y-%m-%d')


# # Extract day of week, month, and year from 'DATE_OCC'
# data['Day_of_Week'] = data['DATE_OCC'].dt.dayofweek
# data['Month'] = data['DATE_OCC'].dt.month
# data['Year'] = data['DATE_OCC'].dt.year

# # Drop the original 'Date_Rptd' and 'DATE_OCC' columns
# data = data.drop(['Date_Rptd', 'DATE_OCC'], axis=1)

# # Convert 'Vict_Sex' and 'Vict_Descent' to dummy variables
# categorical_to_convert = ['Vict_Sex', 'Region_Ethnic_Origin', 'AREA']
# data = pd.get_dummies(data, columns=categorical_to_convert, drop_first=True)
# data['Day_of_Week'] = data['Day_of_Week'].astype('category')
# data['Month'] = data['Month'].astype('category')
# data['Year'] = data['Year'].astype('category')

# # Create dummy variables for these columns
# data = pd.get_dummies(data, columns=['Day_of_Week', 'Month', 'Year'], drop_first=True)


# # We will not convert 'LOCATION' due to its high cardinality
# data.drop(['LOCATION', 'Vict_Descent', 'DR_NO'], axis=1, inplace=True)

In [4]:
def calculate_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((column < lower_bound) | (column > upper_bound)).sum()

In [5]:
statistics = data.describe()

In [6]:
full_statistics = data.describe(include='all')

In [7]:
clean_df

,DR_NO,Date_Rptd,DATE_OCC,TIME_OCC,AREA,AREA_NAME,Rpt_Dist_No,Part_1-2,Crm_Cd,Crm_Cd_Desc,...,Avg_Temp,Avg_Dewpoint,Avg_Humidity,Avg_Windspeed,Avg_Pressure,Total_Precipitation,Crime_Category,Crime_Category_Code,Region_Ethnic_Origin,Weapon_Reported
0,1307355,02/20/2010 12:00:00 AM,2010-02-20,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,56.1,47.5,73.2,9.7,29.8,0.0,Violations and Restraining Orders,7,Hispanic/Latin/Mexican,0
1,100100638,02/20/2010 12:00:00 AM,2010-02-20,100,1,Central,164,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,56.1,47.5,73.2,9.7,29.8,0.0,Vandalism and Property Damage,6,Other,0
2,100100639,02/20/2010 12:00:00 AM,2010-02-20,230,1,Central,164,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,...,56.1,47.5,73.2,9.7,29.8,0.0,"Assault, Battery and Other Violent",0,Hispanic/Latin/Mexican,1
3,100106650,02/20/2010 12:00:00 AM,2010-02-20,45,1,Central,155,2,624,BATTERY - SIMPLE ASSAULT,...,56.1,47.5,73.2,9.7,29.8,0.0,"Assault, Battery and Other Violent",0,Black,1
4,100106654,02/20/2010 12:00:00 AM,2010-02-20,1130,1,Central,146,2,946,OTHER MISCELLANEOUS CRIME,...,56.1,47.5,73.2,9.7,29.8,0.0,Miscellaneous Crimes,2,Hispanic/Latin/Mexican,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883797,220414089,10/08/2022 12:00:00 AM,2022-10-06,900,4,Hollenbeck,406,1,310,BURGLARY,...,69.4,62.9,79.5,6.4,29.9,0.0,Theft and Burglary,5,White,0
2883798,220122081,10/07/2022 12:00:00 AM,2022-10-06,2300,1,Central,176,1,330,BURGLARY FROM VEHICLE,...,69.4,62.9,79.5,6.4,29.9,0.0,Theft and Burglary,5,White,0
2883799,220414017,10/06/2022 12:00:00 AM,2022-10-06,1800,4,Hollenbeck,443,1,330,BURGLARY FROM VEHICLE,...,69.4,62.9,79.5,6.4,29.9,0.0,Theft and Burglary,5,Asian,1
2883800,220318042,10/07/2022 12:00:00 AM,2022-10-06,1330,3,Southwest,374,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,69.4,62.9,79.5,6.4,29.9,0.0,Theft and Burglary,5,Black,0


In [8]:
statistics

,DR_NO,TIME_OCC,AREA,Rpt_Dist_No,Part_1-2,Crm_Cd,Vict_Age,LAT,LON,Avg_Temp,Avg_Dewpoint,Avg_Humidity,Avg_Windspeed,Avg_Pressure,Total_Precipitation,Crime_Category_Code,Weapon_Reported
count,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06
mean,1.620472e+08,1.355479e+03,1.082076e+01,1.128493e+03,1.438238e+00,5.058179e+02,3.119061e+01,3.407727e+01,-1.183572e+02,6.306693e+01,5.130804e+01,6.938296e+01,7.135694e+00,2.983052e+01,2.355219e-02,3.033282e+00,3.384445e-01
std,4.198816e+07,6.483654e+02,6.039248e+00,6.038993e+02,4.961709e-01,2.101850e+02,2.093003e+01,1.143011e-01,1.056891e-01,6.048264e+00,1.019059e+01,1.591146e+01,2.222310e+00,1.845722e-01,1.607327e-01,2.317905e+00,4.731806e-01
min,8.170000e+02,1.000000e+00,1.000000e+00,1.000000e+02,1.000000e+00,1.100000e+02,0.000000e+00,3.334270e+01,-1.188279e+02,4.680000e+01,7.900000e+00,1.250000e+01,0.000000e+00,2.750000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.218202e+08,9.210000e+02,6.000000e+00,6.290000e+02,1.000000e+00,3.300000e+02,1.900000e+01,3.401360e+01,-1.184345e+02,5.860000e+01,4.640000e+01,6.450000e+01,5.700000e+00,2.980000e+01,0.000000e+00,1.000000e+00,0.000000e+00
50%,1.609124e+08,1.430000e+03,1.100000e+01,1.143000e+03,1.000000e+00,4.420000e+02,3.100000e+01,3.406180e+01,-1.183272e+02,6.280000e+01,5.360000e+01,7.400000e+01,6.900000e+00,2.980000e+01,0.000000e+00,3.000000e+00,0.000000e+00
75%,1.912139e+08,1.900000e+03,1.600000e+01,1.638000e+03,2.000000e+00,6.260000e+02,4.600000e+01,3.417210e+01,-1.182761e+02,6.730000e+01,5.860000e+01,7.970000e+01,8.100000e+00,2.990000e+01,0.000000e+00,5.000000e+00,1.000000e+00
max,9.102204e+08,2.359000e+03,2.100000e+01,2.199000e+03,2.000000e+00,9.560000e+02,9.900000e+01,3.479070e+01,-1.176596e+02,8.690000e+01,6.970000e+01,9.820000e+01,2.610000e+01,3.030000e+01,3.000000e+00,7.000000e+00,1.000000e+00


In [9]:
full_statistics

,DR_NO,Date_Rptd,DATE_OCC,TIME_OCC,AREA,AREA_NAME,Rpt_Dist_No,Part_1-2,Crm_Cd,Crm_Cd_Desc,...,Avg_Temp,Avg_Dewpoint,Avg_Humidity,Avg_Windspeed,Avg_Pressure,Total_Precipitation,Crime_Category,Crime_Category_Code,Region_Ethnic_Origin,Weapon_Reported
count,2.883802e+06,2883802,2883802,2.883802e+06,2.883802e+06,2883802,2.883802e+06,2.883802e+06,2.883802e+06,2883802,...,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2.883802e+06,2883802,2.883802e+06,2883802,2.883802e+06
unique,NaN,5129,5111,NaN,NaN,21,NaN,NaN,NaN,144,...,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,6,NaN
top,NaN,02/02/2023 12:00:00 AM,2011-01-01,NaN,NaN,77th Street,NaN,NaN,NaN,BATTERY - SIMPLE ASSAULT,...,NaN,NaN,NaN,NaN,NaN,NaN,Theft and Burglary,NaN,Hispanic/Latin/Mexican,NaN
freq,NaN,1571,4031,NaN,NaN,186392,NaN,NaN,NaN,255465,...,NaN,NaN,NaN,NaN,NaN,NaN,935482,NaN,964118,NaN
mean,1.620472e+08,NaN,NaN,1.355479e+03,1.082076e+01,NaN,1.128493e+03,1.438238e+00,5.058179e+02,NaN,...,6.306693e+01,5.130804e+01,6.938296e+01,7.135694e+00,2.983052e+01,2.355219e-02,NaN,3.033282e+00,NaN,3.384445e-01
std,4.198816e+07,NaN,NaN,6.483654e+02,6.039248e+00,NaN,6.038993e+02,4.961709e-01,2.101850e+02,NaN,...,6.048264e+00,1.019059e+01,1.591146e+01,2.222310e+00,1.845722e-01,1.607327e-01,NaN,2.317905e+00,NaN,4.731806e-01
min,8.170000e+02,NaN,NaN,1.000000e+00,1.000000e+00,NaN,1.000000e+02,1.000000e+00,1.100000e+02,NaN,...,4.680000e+01,7.900000e+00,1.250000e+01,0.000000e+00,2.750000e+01,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00
25%,1.218202e+08,NaN,NaN,9.210000e+02,6.000000e+00,NaN,6.290000e+02,1.000000e+00,3.300000e+02,NaN,...,5.860000e+01,4.640000e+01,6.450000e+01,5.700000e+00,2.980000e+01,0.000000e+00,NaN,1.000000e+00,NaN,0.000000e+00
50%,1.609124e+08,NaN,NaN,1.430000e+03,1.100000e+01,NaN,1.143000e+03,1.000000e+00,4.420000e+02,NaN,...,6.280000e+01,5.360000e+01,7.400000e+01,6.900000e+00,2.980000e+01,0.000000e+00,NaN,3.000000e+00,NaN,0.000000e+00
75%,1.912139e+08,NaN,NaN,1.900000e+03,1.600000e+01,NaN,1.638000e+03,2.000000e+00,6.260000e+02,NaN,...,6.730000e+01,5.860000e+01,7.970000e+01,8.100000e+00,2.990000e+01,0.000000e+00,NaN,5.000000e+00,NaN,1.000000e+00


In [10]:
transposed_statistics = statistics.T

In [11]:
transposed_statistics

,count,mean,std,min,25%,50%,75%,max
DR_NO,2883802.0,1.620472e+08,4.198816e+07,817.0000,1.218202e+08,1.609124e+08,1.912139e+08,9.102204e+08
TIME_OCC,2883802.0,1.355479e+03,6.483654e+02,1.0000,9.210000e+02,1.430000e+03,1.900000e+03,2.359000e+03
AREA,2883802.0,1.082076e+01,6.039248e+00,1.0000,6.000000e+00,1.100000e+01,1.600000e+01,2.100000e+01
Rpt_Dist_No,2883802.0,1.128493e+03,6.038993e+02,100.0000,6.290000e+02,1.143000e+03,1.638000e+03,2.199000e+03
Part_1-2,2883802.0,1.438238e+00,4.961709e-01,1.0000,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00
Crm_Cd,2883802.0,5.058179e+02,2.101850e+02,110.0000,3.300000e+02,4.420000e+02,6.260000e+02,9.560000e+02
Vict_Age,2883802.0,3.119061e+01,2.093003e+01,0.0000,1.900000e+01,3.100000e+01,4.600000e+01,9.900000e+01
LAT,2883802.0,3.407727e+01,1.143011e-01,33.3427,3.401360e+01,3.406180e+01,3.417210e+01,3.479070e+01
LON,2883802.0,-1.183572e+02,1.056891e-01,-118.8279,-1.184345e+02,-1.183272e+02,-1.182761e+02,-1.176596e+02
Avg_Temp,2883802.0,6.306693e+01,6.048264e+00,46.8000,5.860000e+01,6.280000e+01,6.730000e+01,8.690000e+01


In [12]:
transposed_full_statistics = full_statistics.T

In [13]:
transposed_full_statistics

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
DR_NO,2883802.0,NaN,NaN,NaN,162047150.429177,41988156.083138,817.0,121820191.25,160912392.5,191213891.75,910220366.0
Date_Rptd,2883802,5129,02/02/2023 12:00:00 AM,1571,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DATE_OCC,2883802,5111,2011-01-01,4031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TIME_OCC,2883802.0,NaN,NaN,NaN,1355.479162,648.365367,1.0,921.0,1430.0,1900.0,2359.0
AREA,2883802.0,NaN,NaN,NaN,10.820762,6.039248,1.0,6.0,11.0,16.0,21.0
AREA_NAME,2883802,21,77th Street,186392,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rpt_Dist_No,2883802.0,NaN,NaN,NaN,1128.493486,603.899348,100.0,629.0,1143.0,1638.0,2199.0
Part_1-2,2883802.0,NaN,NaN,NaN,1.438238,0.496171,1.0,1.0,1.0,2.0,2.0
Crm_Cd,2883802.0,NaN,NaN,NaN,505.817913,210.184967,110.0,330.0,442.0,626.0,956.0
Crm_Cd_Desc,2883802,144,BATTERY - SIMPLE ASSAULT,255465,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
continuous_vars = data.select_dtypes(include=['int64', 'float64']).columns
categorical_vars = data.select_dtypes(include=['object', 'category']).columns
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [15]:
continuous_stats = data[continuous_vars].describe().transpose()
print("Continuous Variables Descriptive Statistics:")
print(continuous_stats)

Continuous Variables Descriptive Statistics:
                          count          mean          std      min   
DR_NO               2883802.000 162047150.429 41988156.083  817.000  \
TIME_OCC            2883802.000      1355.479      648.365    1.000   
AREA                2883802.000        10.821        6.039    1.000   
Rpt_Dist_No         2883802.000      1128.493      603.899  100.000   
Part_1-2            2883802.000         1.438        0.496    1.000   
Crm_Cd              2883802.000       505.818      210.185  110.000   
Vict_Age            2883802.000        31.191       20.930    0.000   
LAT                 2883802.000        34.077        0.114   33.343   
LON                 2883802.000      -118.357        0.106 -118.828   
Avg_Temp            2883802.000        63.067        6.048   46.800   
Avg_Dewpoint        2883802.000        51.308       10.191    7.900   
Avg_Humidity        2883802.000        69.383       15.911   12.500   
Avg_Windspeed       2883802.000 

In [16]:
categorical_stats = pd.DataFrame(index=categorical_vars, columns=['Count', 'Unique', 'Top', 'Frequency', '% Missing'])

for var in categorical_vars:
    categorical_stats.loc[var, 'Count'] = data[var].count()
    categorical_stats.loc[var, 'Unique'] = data[var].nunique()
    categorical_stats.loc[var, 'Top'] = data[var].mode()[0]
    categorical_stats.loc[var, 'Frequency'] = data[var].value_counts().iloc[0]
    # Calculate the percentage of missing values for each categorical variable
    categorical_stats.loc[var, '% Missing'] = 100 * data[var].isnull().sum() / len(data)

print("Categorical Variables Descriptive Statistics:")
print(categorical_stats)

Categorical Variables Descriptive Statistics:
                        Count Unique                              Top   
Date_Rptd             2883802   5129           02/02/2023 12:00:00 AM  \
DATE_OCC              2883802   5111                       2011-01-01   
AREA_NAME             2883802     21                      77th Street   
Crm_Cd_Desc           2883802    144         BATTERY - SIMPLE ASSAULT   
Vict_Sex              2883802      3                                M   
Vict_Descent          2883802     21                                H   
LOCATION              2883802  80242  6TH                          ST   
Crime_Category        2883802      8               Theft and Burglary   
Region_Ethnic_Origin  2883802      6           Hispanic/Latin/Mexican   

                     Frequency % Missing  
Date_Rptd                 1571     0.000  
DATE_OCC                  4031     0.000  
AREA_NAME               186392     0.000  
Crm_Cd_Desc             255465     0.000  
Vict_Sex

In [17]:
continuous_vars = data.select_dtypes(include=['int64', 'float64']).columns
outliers_count = {col: calculate_outliers(data[col]) for col in continuous_vars}

# Adding the number of outliers to the descriptive statistics table
for col in continuous_vars:
    transposed_statistics.loc[col, 'Outliers'] = outliers_count[col]

In [18]:
transposed_statistics['Skewness'] = data[continuous_vars].skew()
transposed_statistics['Kurtosis'] = data[continuous_vars].kurtosis()
transposed_statistics['Missing Values'] = data[continuous_vars].isnull().sum()

In [19]:
# transposed_statistics['Skewness'] = data.skew()
# transposed_statistics['Kurtosis'] = data.kurtosis()
# transposed_statistics['Missing Values'] = data.isnull().sum()

# # Add Outliers (using the previously defined calculate_outliers function)
# transposed_statistics['Outliers'] = transposed_statistics.index.map(lambda x: calculate_outliers(data[x]) if x in continuous_vars else None)

# # Setting display format to avoid scientific notation
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

# print(transposed_statistics)

In [20]:
numeric_cols = data.select_dtypes(include=[np.number]).columns

# Calculate descriptive statistics for numeric columns only
numeric_stats = data[numeric_cols].describe().transpose()

# Add skewness and kurtosis
numeric_stats['Skewness'] = data[numeric_cols].skew()
numeric_stats['Kurtosis'] = data[numeric_cols].kurtosis()
numeric_stats['% Missing'] = 100 * clean_df[numeric_cols].isnull().sum() / len(clean_df)


# Function to calculate outliers
def calculate_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return ((series < lower_bound) | (series > upper_bound)).sum()

# Add number of outliers
numeric_stats['Outliers'] = data[numeric_cols].apply(calculate_outliers)

# Display the adjusted statistics table
print("Adjusted Numeric Variables Descriptive Statistics:")
print(numeric_stats)

Adjusted Numeric Variables Descriptive Statistics:
                          count          mean          std      min   
DR_NO               2883802.000 162047150.429 41988156.083  817.000  \
TIME_OCC            2883802.000      1355.479      648.365    1.000   
AREA                2883802.000        10.821        6.039    1.000   
Rpt_Dist_No         2883802.000      1128.493      603.899  100.000   
Part_1-2            2883802.000         1.438        0.496    1.000   
Crm_Cd              2883802.000       505.818      210.185  110.000   
Vict_Age            2883802.000        31.191       20.930    0.000   
LAT                 2883802.000        34.077        0.114   33.343   
LON                 2883802.000      -118.357        0.106 -118.828   
Avg_Temp            2883802.000        63.067        6.048   46.800   
Avg_Dewpoint        2883802.000        51.308       10.191    7.900   
Avg_Humidity        2883802.000        69.383       15.911   12.500   
Avg_Windspeed       288380

In [22]:
# Export numeric variables descriptive statistics to CSV
numeric_stats.to_csv('numeric_stats.csv', index=True)

# Export categorical variables descriptive statistics to CSV
categorical_stats.to_csv('categorical_stats.csv', index=True)
